

> Mount Google Drive to Google Colab



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


   Import Libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Conv1D,MaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
# To count the iterations 
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from keras import optimizers
nltk.download('wordnet')

%matplotlib inline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Load training file

In [3]:
# Load training file
df = pd.read_csv("/content/drive/My Drive/recommender/train.csv")
df.head()


,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1


In [4]:
# Drop columns from the training file
df.drop(['review_id', 'title', 'year'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17494 entries, 0 to 17493
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_review      17494 non-null  object
 1   user_suggestion  17494 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 273.5+ KB


In [0]:
# Separate input and output data
X = df.user_review
Y = df.user_suggestion

In [0]:
# Split training data
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [0]:
# Tokenize reviews
max_words = 5000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
# create RNN network
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
# create CNN network
def CNN():
    # create the model
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Conv1D(filters=32, kernel_size=3, activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Activation('relu')(layer)
    layer = LSTM(64)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [13]:
# Compile the model using the best optimizer for CNN
model = CNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=optimizers.Adadelta(learning_rate=1.0, rho=0.95),metrics=['accuracy'])
#optimizers.Adadelta(learning_rate=1.0, rho=0.95)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 50)           250000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 198, 32)           4832      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 99, 32)            0         
_________________________________________________________________
activation_3 (Activation)    (None, 99, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0   

In [14]:
# Compile the model using the best optimizer for RNN
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 200, 50)           250000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [15]:
# Train the model
model.fit(sequences_matrix,Y_train,batch_size=24,epochs=20,
          validation_split=0.15,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)]

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 11895 samples, validate on 2100 samples
Epoch 1/20
11895/11895 [==============================] - 58s 5ms/step - loss: 0.4744 - accuracy: 0.7712 - val_loss: 0.3596 - val_accuracy: 0.8486
Epoch 2/20
11895/11895 [==============================] - 59s 5ms/step - loss: 0.3281 - accuracy: 0.8660 - val_loss: 0.3776 - val_accuracy: 0.8357


In [17]:
# Convert X_test data to sequences for evaluation
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)


3499/3499 [==============================] - 3s 814us/step


In [18]:
print('Loss: {:0.3f}'.format(accr[0]))
print('Accuracy: {:0.3f}'.format(accr[1]))


Loss: 0.375
Accuracy: 0.844


In [0]:
# Prediction of trained model for X_test data
pred1 = model.predict(test_sequences_matrix)

pred = []
for i in pred1:
  if i > 0.5:
    pred.append(1)
  else:
    pred.append(0)

In [20]:
!pip install h5py

In [0]:
# save model to single file
model.save('lstm_model.h5')

In [22]:
#Load unseen data
dfT = pd.read_csv("/content/drive/My Drive/recommender/test.csv")
dfT.head()

,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...


In [23]:
# Sequence matrix of the text data
test_sequencesT = tok.texts_to_sequences(dfT.user_review)
test_sequences_matrixT = sequence.pad_sequences(test_sequencesT,maxlen=max_len)
test_sequences_matrixT

array([[   0,    0,    0, ...,   16,    1,  161],
       [   0,    0,    0, ...,   17,  153,  689],
       [  17,   73, 1884, ...,  701, 1713,  855],
       ...,
       [   0,    0,    0, ..., 2127,   43,    3],
       [   0,    0,    0, ..., 1572, 3915,   81],
       [   0,    0,    0, ...,  744, 1406,   84]], dtype=int32)

In [0]:
# Prediction
prediction = model.predict(test_sequences_matrixT, verbose=0)

In [0]:
predT = []
for i in prediction:
  if i >= 0.7:
    predT.append(1)
  else:
    predT.append(0)

In [0]:
dict1 = {'review_id':dfT['review_id'],'user_suggestion':predT}
dict1
dfTT = pd.DataFrame(dict1)
dfTT.to_csv('/content/drive/My Drive/submission_LSTM_CNN.csv')